In [33]:
# -*- coding: utf-8 -*-
import json
import numpy as np
import pandas as pd
import time, datetime
BUY = 0
SELL = 1

In [58]:
import pymysql
## 加上字符集参数，防止中文乱码
dbconn=pymysql.connect(
  host="127.0.0.1",
  database="huobipro",
  user="root",
  password="123456",
  port=3306,
  charset='utf8'
 )

result = list()
#sql语句
cursor = dbconn.cursor()
for i in range(135000):
    sqlcmd="select value from market_htusdt_trade_detail where id=%d"%(i+1)
    cursor.execute(sqlcmd)
    # 获取剩余结果的第一行数据
    row_1 = cursor.fetchone()
    result.append(row_1[0])
dbconn.close()


In [59]:
# f = open('./data/market.htusdt.trade.detail','r')#以读方式打开文件  
# result = list()  
# for line in f.readlines():                          #依次读取每行  
#     line = line.strip()                             #去掉每行头尾空白  
#     if not len(line) or line.startswith('#'):       #判断是否是空行或注释行  
#         continue                                    #是的话，跳过不处理  
#     result.append(line)                             #保存
# print result 

In [60]:
print len(result)

135000


In [61]:
allTrade = []
for i in range(len(result)):
    trade = json.loads(result[i])
    for j in range(len(trade['tick']['data'])):  
        oneTrade = []
        d = trade['tick']['data'][j]
        oneTrade.append(d['id'])
        oneTrade.append(d['price'])
        oneTrade.append(d['amount'])
        oneTrade.append(d['direction'])
        oneTrade.append(d['ts'])
        allTrade.append(oneTrade)
    


In [62]:

tradeDF = pd.DataFrame(allTrade)
tradeDF.columns = ['id','price','amount','direction','ts']
print tradeDF.head(20)

                      id   price  amount direction             ts
0   76440479664661950000  5.3602   45.48      sell  1526723353833
1   76440489004661960000  5.3602  454.24      sell  1526723355269
2   76440500014661960000  5.3602  431.35      sell  1526723356833
3   76440509104661950000  5.3602  103.23      sell  1526723358219
4   76440516864661960000  5.3602   16.78      sell  1526723359265
5   76440533544661960000  5.3602  413.38      sell  1526723361476
6   76440554684661960000  5.3602  137.58      sell  1526723364208
7   76440558594661560000  5.3603   10.00       buy  1526723364620
8   76440558594661420000  5.3608   75.00       buy  1526723364620
9   76440558594661960000  5.3627    0.12       buy  1526723364620
10  76440558594661960000  5.3627    1.00       buy  1526723364620
11  76440558594661420000  5.3628   75.00       buy  1526723364620
12  76440567844661970000  5.3628  191.30      sell  1526723365748
13  76440570014661970000  5.3628  249.04      sell  1526723366022
14  764405

In [77]:
# 使用time
def millisecondToMinute(s):
    timeStamp = s/1000
    timeArray = time.localtime(timeStamp)
    otherStyleTime = time.strftime("%Y%m%d%H%M", timeArray)
    return int(otherStyleTime)
tradeDF['minute'] = tradeDF['ts'].map(millisecondToMinute)
print tradeDF

                          id   price   amount direction             ts  \
0       76440479664661950000  5.3602    45.48      sell  1526723353833   
1       76440489004661960000  5.3602   454.24      sell  1526723355269   
2       76440500014661960000  5.3602   431.35      sell  1526723356833   
3       76440509104661950000  5.3602   103.23      sell  1526723358219   
4       76440516864661960000  5.3602    16.78      sell  1526723359265   
5       76440533544661960000  5.3602   413.38      sell  1526723361476   
6       76440554684661960000  5.3602   137.58      sell  1526723364208   
7       76440558594661560000  5.3603    10.00       buy  1526723364620   
8       76440558594661420000  5.3608    75.00       buy  1526723364620   
9       76440558594661960000  5.3627     0.12       buy  1526723364620   
10      76440558594661960000  5.3627     1.00       buy  1526723364620   
11      76440558594661420000  5.3628    75.00       buy  1526723364620   
12      76440567844661970000  5.3628  

In [78]:

def norepeatNum(group):#不重复的个数
    return len(set(list(group)))

dictMaping = {
    'id':[norepeatNum],
    'price':['max','min','mean','median','std'],
    'amount':['max','min','mean','median','std','count']
}
newTradeDF = tradeDF.groupby(['minute','direction']).agg(dictMaping)
print newTradeDF.index#apply(lambda x : 0 if str(x)=='buy' else 1)
print newTradeDF.shape

MultiIndex(levels=[[201805191749, 201805191750, 201805191751, 201805191752, 201805191753, 201805191754, 201805191755, 201805191756, 201805191757, 201805191758, 201805191759, 201805191800, 201805191801, 201805191802, 201805191803, 201805191804, 201805191805, 201805191806, 201805191807, 201805191808, 201805191809, 201805191810, 201805191811, 201805191812, 201805191813, 201805191814, 201805191815, 201805191816, 201805191817, 201805191818, 201805191819, 201805191820, 201805191821, 201805191822, 201805191823, 201805191824, 201805191825, 201805191826, 201805191827, 201805191828, 201805191829, 201805191830, 201805191831, 201805191832, 201805191833, 201805191834, 201805191835, 201805191836, 201805191837, 201805191838, 201805191839, 201805191840, 201805191841, 201805191842, 201805191843, 201805191844, 201805191845, 201805191846, 201805191847, 201805191848, 201805191849, 201805191850, 201805191851, 201805191852, 201805191853, 201805191854, 201805191855, 201805191856, 201805191857, 201805191858, 

In [79]:
newTradeDF['direction'] = range(newTradeDF.shape[0])
newTradeDF['direction'] = newTradeDF['direction'].map(lambda x : BUY if newTradeDF.index[x][1]=='buy' else SELL)
newTradeDF['minute']= range(newTradeDF.shape[0])
newTradeDF['minute'] = newTradeDF['minute'].map(lambda x : newTradeDF.index[x][0])
newTradeDF.reset_index(drop=True, inplace=True)
print newTradeDF

            amount                                                          \
               max         min         mean      median          std count   
0      3285.040000    0.120000   391.598182   75.000000   967.305813    11   
1       454.240000    6.510000   197.697368  199.980000   155.780115    19   
2      2000.000000    0.100000   256.044737   64.150000   526.089843    19   
3       372.710000    1.000000   123.978947   79.700000   117.318976    19   
4      3188.880000    0.650000   595.785455  251.645000   849.180098    22   
5      1860.340000    2.780000   272.532568  243.650000   314.830476    74   
6      1911.125786   10.000000   200.076399   43.720000   516.243170    13   
7      6009.310000    1.170000   416.592721  243.700000   873.596570    53   
8      1607.280000    5.000000   327.282857  107.830000   531.564003    14   
9      3349.830000   12.320000   386.420500  116.600000   752.726457    20   
10      163.020000    1.000000    69.006667   43.000000    84.08

In [80]:
print newTradeDF.columns

MultiIndex(levels=[[u'amount', u'price', u'id', u'direction', u'minute'], [u'count', u'max', u'mean', u'median', u'min', u'norepeatNum', u'std', u'']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3, 4], [1, 4, 2, 3, 6, 0, 1, 4, 2, 3, 6, 5, 7, 7]])


In [81]:
newColumns = []
for i in range(len(newTradeDF.columns)):
    if newTradeDF.columns[i][1]:
        newColumns.append(str(newTradeDF.columns[i][0])+'_'+newTradeDF.columns[i][1])
    else:        
        newColumns.append(str(newTradeDF.columns[i][0]))
newTradeDF.columns = newColumns

['amount_max', 'amount_min', 'amount_mean', 'amount_median', 'amount_std', 'amount_count', 'price_max', 'price_min', 'price_mean', 'price_median', 'price_std', 'id_norepeatNum', 'direction', 'minute']


In [82]:
print newTradeDF

        amount_max  amount_min  amount_mean  amount_median   amount_std  \
0      3285.040000    0.120000   391.598182      75.000000   967.305813   
1       454.240000    6.510000   197.697368     199.980000   155.780115   
2      2000.000000    0.100000   256.044737      64.150000   526.089843   
3       372.710000    1.000000   123.978947      79.700000   117.318976   
4      3188.880000    0.650000   595.785455     251.645000   849.180098   
5      1860.340000    2.780000   272.532568     243.650000   314.830476   
6      1911.125786   10.000000   200.076399      43.720000   516.243170   
7      6009.310000    1.170000   416.592721     243.700000   873.596570   
8      1607.280000    5.000000   327.282857     107.830000   531.564003   
9      3349.830000   12.320000   386.420500     116.600000   752.726457   
10      163.020000    1.000000    69.006667      43.000000    84.082579   
11     3250.310000    1.000000   990.004167     721.850000  1027.121044   
12     2466.180000    1.0

In [83]:
buyDF = newTradeDF[newTradeDF['direction']==BUY]
sellDF = newTradeDF[newTradeDF['direction']==SELL]
buyDF = buyDF.drop(['direction'], axis=1)
sellDF = sellDF.drop(['direction'], axis=1)

buyDF.columns = [ 'buy_'+columnName for columnName in buyDF.columns]
sellDF.columns = [ 'sell_'+columnName for columnName in sellDF.columns]
buySellDF = buyDF.merge(sellDF, left_on='buy_minute', right_on='sell_minute')
buySellDF = buySellDF.drop(['buy_minute'], axis=1)
print buySellDF.columns



Index([u'buy_amount_max', u'buy_amount_min', u'buy_amount_mean',
       u'buy_amount_median', u'buy_amount_std', u'buy_amount_count',
       u'buy_price_max', u'buy_price_min', u'buy_price_mean',
       u'buy_price_median', u'buy_price_std', u'buy_id_norepeatNum',
       u'sell_amount_max', u'sell_amount_min', u'sell_amount_mean',
       u'sell_amount_median', u'sell_amount_std', u'sell_amount_count',
       u'sell_price_max', u'sell_price_min', u'sell_price_mean',
       u'sell_price_median', u'sell_price_std', u'sell_id_norepeatNum',
       u'sell_minute'],
      dtype='object')


In [84]:
buySellDF.rename(columns={'sell_minute':'minute'}, inplace=True)
buySellDF.set_index(buySellDF['minute'], inplace=True, drop=True)
buySellDF = buySellDF.drop(['minute'], axis=1)
print buySellDF.columns
print buySellDF.index

Index([u'buy_amount_max', u'buy_amount_min', u'buy_amount_mean',
       u'buy_amount_median', u'buy_amount_std', u'buy_amount_count',
       u'buy_price_max', u'buy_price_min', u'buy_price_mean',
       u'buy_price_median', u'buy_price_std', u'buy_id_norepeatNum',
       u'sell_amount_max', u'sell_amount_min', u'sell_amount_mean',
       u'sell_amount_median', u'sell_amount_std', u'sell_amount_count',
       u'sell_price_max', u'sell_price_min', u'sell_price_mean',
       u'sell_price_median', u'sell_price_std', u'sell_id_norepeatNum'],
      dtype='object')
Int64Index([201805191749, 201805191750, 201805191751, 201805191752,
            201805191753, 201805191754, 201805191755, 201805191756,
            201805191757, 201805191758,
            ...
            201805221352, 201805221353, 201805221354, 201805221355,
            201805221356, 201805221357, 201805221358, 201805221359,
            201805221400, 201805221401],
           dtype='int64', name=u'minute', length=3551)


In [85]:
buySellDF['buysell_amount_mean'] = buySellDF['buy_amount_mean']-buySellDF['sell_amount_mean']
buySellDF['buysell_amount_median'] = buySellDF['buy_amount_median']-buySellDF['sell_amount_median']
buySellDF['buysell_amount_std'] = buySellDF['buy_amount_std']-buySellDF['sell_amount_std']
buySellDF['buysell_amount_count'] = buySellDF['buy_amount_count']-buySellDF['sell_amount_count']
buySellDF['buysell_price_mean'] = buySellDF['buy_price_mean']-buySellDF['sell_price_mean']
buySellDF['buysell_price_median'] = buySellDF['buy_price_median']-buySellDF['sell_price_median']
buySellDF['buysell_price_std'] = buySellDF['buy_price_std']-buySellDF['sell_price_std']
buySellDF['buysell_id_norepeatNum'] = buySellDF['buy_id_norepeatNum']-buySellDF['sell_id_norepeatNum']

In [86]:
print buySellDF.columns

Index([u'buy_amount_max', u'buy_amount_min', u'buy_amount_mean',
       u'buy_amount_median', u'buy_amount_std', u'buy_amount_count',
       u'buy_price_max', u'buy_price_min', u'buy_price_mean',
       u'buy_price_median', u'buy_price_std', u'buy_id_norepeatNum',
       u'sell_amount_max', u'sell_amount_min', u'sell_amount_mean',
       u'sell_amount_median', u'sell_amount_std', u'sell_amount_count',
       u'sell_price_max', u'sell_price_min', u'sell_price_mean',
       u'sell_price_median', u'sell_price_std', u'sell_id_norepeatNum',
       u'buysell_amount_mean', u'buysell_amount_median', u'buysell_amount_std',
       u'buysell_amount_count', u'buysell_price_mean', u'buysell_price_median',
       u'buysell_price_std', u'buysell_id_norepeatNum'],
      dtype='object')


In [87]:
indexList = list(buySellDF.index)        
print indexList

print buySellDF.columns

[201805191749L, 201805191750L, 201805191751L, 201805191752L, 201805191753L, 201805191754L, 201805191755L, 201805191756L, 201805191757L, 201805191758L, 201805191759L, 201805191800L, 201805191801L, 201805191802L, 201805191803L, 201805191804L, 201805191805L, 201805191806L, 201805191807L, 201805191808L, 201805191809L, 201805191810L, 201805191811L, 201805191812L, 201805191813L, 201805191814L, 201805191815L, 201805191816L, 201805191817L, 201805191818L, 201805191819L, 201805191820L, 201805191821L, 201805191822L, 201805191823L, 201805191824L, 201805191825L, 201805191826L, 201805191827L, 201805191828L, 201805191829L, 201805191830L, 201805191831L, 201805191832L, 201805191833L, 201805191834L, 201805191835L, 201805191836L, 201805191837L, 201805191838L, 201805191839L, 201805191840L, 201805191841L, 201805191842L, 201805191843L, 201805191844L, 201805191845L, 201805191846L, 201805191847L, 201805191848L, 201805191849L, 201805191850L, 201805191851L, 201805191852L, 201805191853L, 201805191854L, 201805191

In [88]:
newBuySell = []
labels = []
for i in range(len(indexList)):    
    if indexList[i]+1 in indexList:
        lineData = []
        for key in buySellDF.loc[indexList[i]].keys():
            lineData.append(buySellDF.loc[indexList[i+1]][key] - buySellDF.loc[indexList[i]][key])
        newBuySell.append(lineData)
        i_price_mean = (buySellDF.loc[indexList[i]]['buy_price_mean'] + buySellDF.loc[indexList[i]]['sell_price_mean'])*0.5
        i1_price_mean = (buySellDF.loc[indexList[i+1]]['buy_price_mean'] + buySellDF.loc[indexList[i+1]]['sell_price_mean'])*0.5
        labels.append(1 if i1_price_mean-i_price_mean>0 else 0)

In [89]:
print len(labels)
num = int(len(labels)*8/10)
print num
testDF = pd.DataFrame(newBuySell)
# for i in range(32):
#     ddd = pd.Series(testDF[i])
#     print len(ddd.isna()==True)
print testDF

3413
2730
                0          1           2           3            4      5   \
0     -1285.040000  -0.020000 -135.553445  -10.850000  -441.215970    8.0   
1      1188.880000   0.550000  339.740718  187.495000   323.090254    3.0   
2     -1277.754214   9.350000 -395.709055 -207.925000  -332.936928   -9.0   
3      -303.845786  -5.000000  127.206458   64.110000    15.320833    1.0   
4     -1444.260000  -4.000000 -258.276190  -64.830000  -447.481424  -11.0   
5      2303.160000   0.000000  131.799524    0.720000   443.967946   18.0   
6     -2151.157680  -0.090000 -102.374173   26.970000  -426.941887   -6.0   
7       595.407680   4.090000  122.640484   35.280000   158.983184   13.0   
8      2279.410000  11.610000  176.920116  -11.200000   464.517422    5.0   
9      -211.880000 -11.610000  180.791690    4.940000   177.350238   -1.0   
10   -45000.000000   0.169299 -556.064269    0.000000 -4008.629898 -191.0   
11    -3759.470000  -0.050000 -263.642554  -31.885000  -613.260916

In [76]:
from sklearn.ensemble import GradientBoostingClassifier
X = pd.DataFrame(newBuySell)
y = pd.Series(labels)
X_train, X_test = X[:num], X[num:]
y_train, y_test = y[:num], y[num:]
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [91]:
##############lightgbm方法################
X = pd.DataFrame(newBuySell)
y = pd.Series(labels)
X_train, X_test = X[:num], X[num:]
y_train, y_test = y[:num], y[num:]

def printScore(booster, data, label):
    preds_sub = booster.predict(data)
    true_sub = list(label)
    preds_sub = map(lambda x : 0 if x<0.5 else 1, list(preds_sub))
    true_sub = map(lambda x : int(x), true_sub)
    scoreList = map(lambda x,y: 1 if x==y else 0, preds_sub, true_sub)
    num = reduce(lambda x,y: x+y, scoreList)
    print('score:%f'%(num/float(len(preds_sub))))
    
# lgb算法
import lightgbm as lgb
import itertools
train = lgb.Dataset(X_train, label=y_train)
test = lgb.Dataset(X_test, label=y_test, reference=train)

num_rounds = [1000]
num_leaves = [2**4]# 决策树叶子节点数
for num_round, num_leave in list(itertools.product(num_rounds, num_leaves)):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression_l2',
        'metric': 'l2',
        # 'objective': 'multiclass',
        # 'metric': 'multi_error',
        # 'num_class':5,
        'min_child_weight': 3,
        'num_leaves': num_leave,
        'lambda_l2': 10,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'colsample_bylevel': 0.7,
        'learning_rate': 0.05,
        'tree_method': 'exact',
        'seed': 2017,
        'nthread': 12,
        'silent': True
    }
    gbm = lgb.train(params,
                    train,
                    num_round,
                    verbose_eval=500,
                    valid_sets=[train, test]
                    )


    print ("\n\n 训练次数：%d, num_leave:%d"%(num_round, num_leave))
    print ("训练集：")
    printScore(gbm, X_train, y_train)
    print ("测试集：")
    printScore(gbm, X_test, y_test)

[500]	training's l2: 0.000890073	valid_1's l2: 0.0110654
[1000]	training's l2: 0.000212546	valid_1's l2: 0.0111585


 训练次数：1000, num_leave:16
训练集：
score:1.000000
测试集：
score:0.991215
